# Exercício 01: Missão de Reconhecimento Aéreo
Um avião de reconhecimento precisa sobrevoar 5 alvos estratégicos em território hostil, partindo e retornando à base aérea. Cada alvo possui um nível de risco associado à presença de defesas antiaéreas.
## Objetivo
Encontrar a rota que minimize o custo total da missão, considerando:
Distância percorrida (consumo de combustível)
Risco acumulado ao sobrevoar cada alvo
## Dados do Problema
### 1. Matriz de Distâncias (km)
     Base  Radar  Ponte  Depósito  Porto  Fábrica
Base    0     45     30      50      65     40
Radar   45    0      55      40      60     35
Ponte   30    55     0       25      40     45
Depósito 50   40     25      0       30     50
Porto   65    60     40      30      0      25
Fábrica 40    35     45      50      25     0
### 2. Níveis de Risco dos Alvos
Base Aérea: 0 (seguro)
Radar Inimigo: 8 (alto risco)
Ponte Estratégica: 3 (baixo risco)
Depósito de Armas: 5 (médio risco)
Porto Militar: 6 (médio risco)
Fábrica de Munições: 4 (baixo risco)
### 3. Especificações da Aeronave
Consumo: 2 litros/km
Capacidade do tanque: 500 litros
Função de Custo
Custo Total = Distância Total + (Soma dos Riscos × 5)
## Tarefas
### Implemente a classe MissaoAerea herdando de ProblemInterface com os métodos:
gerar_individuo(): Gera uma rota aleatória começando na base (índice 0)
calcular_fitness(): Fitness = 1 / Custo Total
crossover(): Implementar crossover preservando a base no início
mutacao(): Implementar mutação sem mover a base


## Execute o algoritmo genético e responda:
### Qual a melhor rota encontrada?
### Qual o custo total dessa rota?
### A missão é viável com o combustível disponível?
### Plote o gráfico de custo.
# Exemplo de Estrutura de Dados
python
## Índices dos locais
INDICES = {
    0: "Base Aérea",
    1: "Radar Inimigo", 
    2: "Ponte Estratégica",
    3: "Depósito de Armas",
    4: "Porto Militar",
    5: "Fábrica de Munições"
}

## Matriz de distâncias
DISTANCIAS = [
    [0,  45, 30, 50, 65, 40],
    [45, 0,  55, 40, 60, 35],
    [30, 55, 0,  25, 40, 45],
    [50, 40, 25, 0,  30, 50],
    [65, 60, 40, 30, 0,  25],
    [40, 35, 45, 50, 25, 0]
]

Níveis de risco
RISCOS = [0, 8, 3, 5, 6, 4]
Dicas
A rota sempre deve começar no índice 0 (Base)
Lembre-se de adicionar o retorno à base no cálculo do custo
Use Order Crossover para manter rotas válidas
Verifique se o combustível necessário não excede a capacidade


In [20]:
import random as rng
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

Matplotlib is building the font cache; this may take a moment.


In [21]:


class ProblemInterface(ABC):
    """Interface que define as operações específicas do problema"""
    
    @abstractmethod
    def gerar_individuo(self):
        """Gera um indivíduo aleatório"""
        pass
    
    @abstractmethod
    def calcular_fitness(self, individuo):
        """Calcula o fitness de um indivíduo"""
        pass
    
    @abstractmethod
    def crossover(self, pai1, pai2):
        """Realiza crossover entre dois pais"""
        pass
    
    @abstractmethod
    def mutacao(self, individuo, taxa_mutacao):
        """Aplica mutação em um indivíduo"""
        pass


In [22]:
INDICES = {
    0: "Base Aérea",
    1: "Radar Inimigo", 
    2: "Ponte Estratégica",
    3: "Depósito de Armas",
    4: "Porto Militar",
    5: "Fábrica de Munições"
}
# Matriz de distâncias
DISTANCIAS = [
    [0,  45, 30, 50, 65, 40],
    [45, 0,  55, 40, 60, 35],
    [30, 55, 0,  25, 40, 45],
    [50, 40, 25, 0,  30, 50],
    [65, 60, 40, 30, 0,  25],
    [40, 35, 45, 50, 25, 0]
]
RISCOS = [0, 8, 3, 5, 6, 4]
CONSUMO_POR_KM = 2
CAPACIDADE_TANQUE = 500

In [49]:
class MissaoAerea(ProblemInterface):
    def __init__ (self,consume,tank_capacity,index,distances,risks):
        self.consumo_por_km = consume
        self.capacidade_tanque = tank_capacity
        self.indices = index
        self.distancias = distances
        self.riscos = risks
        self.chaves = list(index.keys())

    def gerar_individuo(self):
        caminho = list(range(1, len(self.chaves)))  # Alvos (exceto a base)
        rng.shuffle(caminho)
        return [0] + caminho + [0]
   
    def calcular_custo(self, individuo) -> float: #Calcula custo total (distancia_total + 5x risco_total) e a distancia
        distancia_total = 0;
        riscos_soma = 0;
        old = 0;
        for x in individuo:
            distancia_total += self.distancias[old][x]
            riscos_soma += self.riscos[x]
            old = x
        return (distancia_total+(riscos_soma*5)),distancia_total
    def calcular_fitness(self, individuo):
        custo_total, _ = self.calcular_custo(individuo)
        if (custo_total > 0):
            return 1/custo_total
        else:
            return float(inf)
    
    def fill_offspring(self,offspring_target_middle, parent_source_middle, mapping_dict_source_to_target,cut1,cut2,size):
        copied_segment_values = set(offspring_target_middle[cut1:cut2])
        for i in range(size):
            if i < cut1 or i >= cut2:
                gene_padrao = parent_source_middle[i]

                geracao_atual = gene_padrao
                while geracao_atual in copied_segment_values:
                    if geracao_atual in mapping_dict_source_to_target:
                        geracao_atual = mapping_dict_source_to_target[geracao_atual]
                    else:
                        break
                offspring_target_middle[i] = geracao_atual
        return offspring_target_middle
    def crossover(self, parent1, parent2):
        size = len(parent1) - 2 # Exclui a base do início e do fim
        p1_meio = parent1[1:-1]
        p2_meio = parent2[1:-1]

        offspring1_meio = [None] * size
        offspring2_meio = [None] * size

        cut1, cut2 = sorted(rng.sample(range(size), 2))

        offspring1_meio[cut1:cut2] = p1_meio[cut1:cut2]
        offspring2_meio[cut1:cut2] = p2_meio[cut1:cut2]

        map_p1_to_p2 = {p1_meio[i]: p2_meio[i] for i in range(cut1, cut2)}
        map_p2_to_p1 = {p2_meio[i]: p1_meio[i] for i in range(cut1, cut2)}

    
        offspring1_meio = self.fill_offspring(offspring1_meio, p2_meio, map_p2_to_p1,cut1,cut2,size)
        offspring2_meio = self.fill_offspring(offspring2_meio, p1_meio, map_p1_to_p2,cut1,cut2,size)
    
        return [0] + offspring1_meio + [0], [0] + offspring2_meio + [0]
    def mutacao(self, individuo, taxa_mutacao):
        if rng.rng() < taxa_mutacao:
            # Não mutar a base (índice 0 e último)
            indices_mutaveis = list(range(1, len(individuo) - 1))
            if len(indices_mutaveis) >= 2:
                idx1, idx2 = rng.sample(indices_mutaveis, 2)
                individuo[idx1], individuo[idx2] = individuo[idx2], individuo[idx1]
        return individuo

In [50]:
class AlgoritmoGenetico:
    def __init__(self, problema, tamanho_populacao, geracoes, taxa_crossover, taxa_mutacao):
        self.problema = problema
        self.tamanho_populacao = tamanho_populacao
        self.geracoes = geracoes
        self.taxa_crossover = taxa_crossover
        self.taxa_mutacao = taxa_mutacao
        self.melhores_custos_por_geracao = []

    def _selecionar(self, populacao, fitnesses):
        """Seleção por roleta."""
        total_fitness = sum(fitnesses)
        if total_fitness == 0:
            # If all fitness are zero (e.g., very high costs), select randomly
            return random.choice(populacao)
        
        pick = rng.uniform(0, total_fitness)
        current = 0
        for individuo, fitness in zip(populacao, fitnesses):
            current += fitness
            if current > pick:
                return individuo
        return random.choice(populacao) # Fallback

    def executar(self):
        populacao = [self.problema.gerar_individuo() for _ in range(self.tamanho_populacao)]
        melhor_individuo = None
        melhor_custo = float('inf')

        for geracao in range(self.geracoes):
            fitnesses = [self.problema.calcular_fitness(ind) for ind in populacao]
            
            # Encontra o melhor da geração atual
            melhor_fitness_geracao = -1
            melhor_individuo_geracao = None
            for i, individuo in enumerate(populacao):
                if fitnesses[i] > melhor_fitness_geracao:
                    melhor_fitness_geracao = fitnesses[i]
                    melhor_individuo_geracao = individuo
            
            custo_melhor_geracao, _ = self.problema.calcular_custo(melhor_individuo_geracao)
            self.melhores_custos_por_geracao.append(custo_melhor_geracao)

            if custo_melhor_geracao < melhor_custo:
                melhor_custo = custo_melhor_geracao
                melhor_individuo = melhor_individuo_geracao

            nova_populacao = []
            while len(nova_populacao) < self.tamanho_populacao:
                pai1 = self._selecionar(populacao, fitnesses)
                pai2 = self._selecionar(populacao, fitnesses)

                if rng.random() < self.taxa_crossover:
                    filho1, filho2 = self.problema.crossover(pai1, pai2)
                else:
                    filho1, filho2 = pai1, pai2

                filho1 = self.problema.mutacao(filho1, self.taxa_mutacao)
                filho2 = self.problema.mutacao(filho2, self.taxa_mutacao)

                nova_populacao.append(filho1)
                if len(nova_populacao) < self.tamanho_populacao:
                    nova_populacao.append(filho2)

            populacao = nova_populacao

        return melhor_individuo, melhor_custo

In [51]:
TAMANHO_POPULACAO=100
GERACOES=500
TAXA_CROSSOVER=0.8
TAXA_MUTACAO=0.1

def main():
    m1 = MissaoAerea(CONSUMO_POR_KM,CAPACIDADE_TANQUE,INDICES,DISTANCIAS,RISCOS)
    ag = AlgoritmoGenetico(m1,TAMANHO_POPULACAO,GERACOES,TAXA_CROSSOVER,TAXA_MUTACAO)
    print("Resultados da Missao Aerea:")
    melhor_rota_indices, custo_total_rota = ag.executar()
    melhor_rota_nomes = [m1.indices[i] for i in melhor_rota_indices]
    _, distancia_total_rota = m1.calcular_custo(melhor_rota_indices)
    consumo_rota = distancia_total_rota * m1.consumo_por_km
    print(f"Melhor rota encontrada (índices): {melhor_rota_indices}")
    print(f"Melhor rota encontrada (nomes): {melhor_rota_nomes}")
    print(f"Custo total dessa rota: {custo_total_rota:.2f}")
    print(f"Distância total percorrida: {distancia_total_rota:.2f} km")
    print(f"Consumo de combustível da rota: {consumo_rota:.2f} litros")
    print(f"Capacidade do tanque: {m1.capacidade_tanque} litros")
    print(f"A missão é viável com o combustível disponível? {'Sim' if (consumo_rota <= m1.capacidade_tanque) else 'Não'}")
    # Plotar gráfico de custo
    plt.figure(figsize=(10, 6))
    plt.plot(ag.melhores_custos_por_geracao)
    plt.title('Melhor Custo por Geração')
    plt.xlabel('Geração')
    plt.ylabel('Custo Total')
    plt.grid(True)
    plt.show()
main()


Resultados da Missao Aerea:


<class 'NameError'>: name 'size' is not defined